In [7]:
import scipy
import torch
import finufft
import numpy as np

from numpy.random import standard_normal

In [12]:
Ns = [5, 10, 100, 1000, 100000]
case1 = np.array([1.0, 2.5, -1.0, -1.5, 1.5], dtype=np.complex128)
cases = [
    case1,
]

for i in Ns:
    cases.append(standard_normal(i) + 1j * standard_normal(i))

# cases

In [33]:
curr = cases[5]

curr

array([-0.66527073-0.61035547j, -2.12140142-0.5732981j ,
       -1.010923  +1.54094655j, ...,  0.34772392-2.38756294j,
        2.28183424-1.13824207j,  0.37741838-0.27767027j])

In [34]:
curr_tensor = torch.from_numpy(curr)
against_torch = torch.fft.fft(curr_tensor)

In [35]:
N = len(curr)

finufft_out = finufft.nufft1d1(
    2 * np.pi * np.arange(0, 1, 1 / N), curr, N, modeord=1, isign=-1
)

In [36]:
against_torch

tensor([  49.7882-275.3234j, -346.6685+657.1234j, -292.1692+786.4391j,
         ...,  243.0341-404.0582j,  -31.8155+253.3499j,
          33.8828+387.0750j], dtype=torch.complex128)

In [37]:
finufft_out

array([  49.78823074-275.32356997j, -346.66872115+657.12369894j,
       -292.16933971+786.43948763j, ...,  243.03421708-404.0584142j ,
        -31.81552999+253.34999937j,   33.88280688+387.0752464j ])

In [38]:
(torch.from_numpy(finufft_out) - against_torch)

tensor([ 2.6338e-05-0.0001j, -1.8338e-04+0.0003j, -1.5455e-04+0.0004j,
         ...,  1.2856e-04-0.0002j, -1.6830e-05+0.0001j,
         1.7924e-05+0.0002j], dtype=torch.complex128)

In [31]:
np.array([0, 1, 2, 3, 4], dtype=np.complex128)

array([0.+0.j, 1.+0.j, 2.+0.j, 3.+0.j, 4.+0.j])

In [62]:
######################## TYPE 2

curr = cases[0]
N = len(curr)

forward = torch.fft.fft(torch.from_numpy(curr))

against_torch = torch.fft.ifft(forward)

finufft_out = (
    finufft.nufft1d2(
        2 * np.pi * np.arange(0, 1, 1 / N), forward.numpy(), modeord=1, isign=-1
    )
    / N
)

In [63]:
against_torch

tensor([ 1.0000+0.j,  2.5000+0.j, -1.0000+0.j, -1.5000+0.j,  1.5000+0.j],
       dtype=torch.complex128)

In [64]:
finufft_out

array([ 0.99999908-6.45831755e-08j, -2.0225425 -1.46946302e+00j,
       -0.30901668-9.51056451e-01j, -0.46352527+1.42658489e+00j,
       -1.21352556+8.81677767e-01j])

In [66]:
torch.norm(against_torch - finufft_out)

tensor(5.9367, dtype=torch.float64)